# Chicago EUI

In [1]:
import pandas as pd
# The data comes from https://www.chicago.gov/city/en/progs/env/building-energy-benchmarking---transparency.html
# --> https://data.cityofchicago.org/Environment-Sustainable-Development/Chicago-Energy-Benchmarking/xq83-jr8c/about_data
# --> Export


eui_chicago_path = "https://drive.google.com/uc?id=1XYSIKSaMJytSMFGnq9kYJSSq44qv-zGK"
eui_chicago_df = pd.read_csv(eui_chicago_path)
eui_chicago_df = eui_chicago_df.rename(columns={'Latitude': 'latitude', 'Longitude': 'longitude'})



In [2]:
eui_chicago_df.columns

Index(['Data Year', 'ID', 'Property Name', 'Reporting Status', 'Address',
       'ZIP Code', 'Chicago Energy Rating',
       'Exempt From Chicago Energy Rating', 'Community Area',
       'Primary Property Type', 'Gross Floor Area - Buildings (sq ft)',
       'Year Built', '# of Buildings', 'Water Use (kGal)', 'ENERGY STAR Score',
       'Electricity Use (kBtu)', 'Natural Gas Use (kBtu)',
       'District Steam Use (kBtu)', 'District Chilled Water Use (kBtu)',
       'All Other Fuel Use (kBtu)', 'Site EUI (kBtu/sq ft)',
       'Source EUI (kBtu/sq ft)', 'Weather Normalized Site EUI (kBtu/sq ft)',
       'Weather Normalized Source EUI (kBtu/sq ft)',
       'Total GHG Emissions (Metric Tons CO2e)',
       'GHG Intensity (kg CO2e/sq ft)', 'latitude', 'longitude', 'Location',
       'Row_ID'],
      dtype='object')

In [3]:
eui_chicago_df.shape

(28329, 30)

In [4]:
property_type_classification = {
    # Residential
    "Residential": "Residential",
    "Multifamily Housing": "Residential",
    "Multifamily¬†Housing": "Residential",
    "Multifamily Housing": "Residential",
    "Residence Hall/Dormitory": "Residential",
    "Residential Care Facility": "Residential",
    "Senior Care Community": "Residential",
    "Senior Living Community": "Residential",
    "Lodging": "Residential",
    "Hotel": "Residential",
    "Other - Lodging/Residential": "Residential",

    # Non-Residential
    "Adult Education": "Non-Residential",
    "Ambulatory Surgical Center": "Non-Residential",
    "Automobile Dealership": "Non-Residential",
    "Vehicle Dealership": "Non-Residential",
    "Bank Branch": "Non-Residential",
    "College/University": "Non-Residential",
    "Commerce de détail": "Non-Residential",
    "Commerce de detail": "Non-Residential",
    "Commercial": "Non-Residential",
    "Convention Center": "Non-Residential",
    "Courthouse": "Non-Residential",
    "Data Center": "Non-Residential",
    "Distribution Center": "Non-Residential",
    "Enclosed Mall": "Non-Residential",
    "Financial Office": "Non-Residential",
    "Fitness Center/Health Club/Gym": "Non-Residential",
    "Food Service": "Non-Residential",
    "Health Care": "Non-Residential",
    "Hospital (General Medical & Surgical)": "Non-Residential",
    "Ice/Curling Rink": "Non-Residential",
    "Indoor Arena": "Non-Residential",
    "K-12 School": "Non-Residential",
    "Laboratory": "Non-Residential",
    "Library": "Non-Residential",
    "Lifestyle Center": "Non-Residential",
    "Medical Office": "Non-Residential",
    "Mixed Use Property": "Non-Residential",
    "Movie Theater": "Non-Residential",
    "Municipal": "Non-Residential",
    "Museum": "Non-Residential",
    "Office": "Non-Residential",
    "Other": "Non-Residential",
    "Other - Education": "Non-Residential",
    "Other - Entertainment/Public Assembly": "Non-Residential",
    "Other - Mall": "Non-Residential",
    "Other - Public Services": "Non-Residential",
    "Other - Recreation": "Non-Residential",
    "Other - Services": "Non-Residential",
    "Other - Specialty Hospital": "Non-Residential",
    "Outpatient Rehabilitation/Physical Therapy": "Non-Residential",
    "Performing Arts": "Non-Residential",
    "Pre-school/Daycare": "Non-Residential",
    "Prison/Incarceration": "Non-Residential",
    "Repair Services (Vehicle, Shoe, Locksmith, etc.)": "Non-Residential",
    "Retail Store": "Non-Residential",
    "Social/Meeting Hall": "Non-Residential",
    "Stadium (Open)": "Non-Residential",
    "Strip Mall": "Non-Residential",
    "Supermarket/Grocery Store": "Non-Residential",
    "Urgent Care/Clinic/Other Outpatient": "Non-Residential",
    "Wholesale Club/Supercenter": "Non-Residential",
    "Worship Facility": "Non-Residential",
    "Not Available": None, }


eui_chicago_df["Property Type Category"] = eui_chicago_df['Primary Property Type'].map(property_type_classification)
eui_chicago_df.loc[eui_chicago_df['Primary Property Type'].str.contains('Multifamily', case=False, na=False), 'Property Type Category'] = 'Residential'
eui_chicago_df.loc[eui_chicago_df['Primary Property Type'].str.contains('dtail', case=False, na=False), 'Property Type Category'] =  "Non-Residential"


eui_chicago_df.groupby("Property Type Category").size()


# we are goint to delete Property Type Category = Nan and Property Type Category = Other (4000 obs)
eui_chicago_df = eui_chicago_df.dropna(subset=["Primary Property Type"])  
eui_chicago_df = eui_chicago_df[~eui_chicago_df["Primary Property Type"].isin(["Not Available"])] 


In [5]:
#1 kBtu = 0.2931 kWh
#1 ft² = 0.092903 m²

# Conversion factors
kBtu_to_kWh = 0.2931
ft2_to_m2 = 0.092903

eui_chicago_df = eui_chicago_df[eui_chicago_df["Weather Normalized Site EUI (kBtu/sq ft)"] > 0]

eui_chicago_df['EUI (kWh/m²/year)'] = (
    eui_chicago_df['Weather Normalized Site EUI (kBtu/sq ft)'] * kBtu_to_kWh / ft2_to_m2
)

eui_chicago_df["Residential EUI (kWh/m²/year)"] = eui_chicago_df.apply(
    lambda x: x["EUI (kWh/m²/year)"] if x["Property Type Category"] == "Residential" else None, axis=1
)

eui_chicago_df["Non-residential EUI (kWh/m²/year)"] = eui_chicago_df.apply(
    lambda x: x["EUI (kWh/m²/year)"] if x["Property Type Category"] == "Non-Residential" else None, axis=1
)

eui_chicago_df["Residential Area (ft²)"] = eui_chicago_df.apply(
    lambda x: x["Gross Floor Area - Buildings (sq ft)"] if x["Property Type Category"] == "Residential" else None, axis=1
)

eui_chicago_df["Non-residential Area (ft²)"] = eui_chicago_df.apply(
    lambda x: x["Gross Floor Area - Buildings (sq ft)"] if x["Property Type Category"] == "Non-Residential" else None, axis=1
)


In [6]:
eui_chicago_df = eui_chicago_df[[
    'latitude', 'longitude', 
    'Residential EUI (kWh/m²/year)', 'Non-residential EUI (kWh/m²/year)', 
    'Residential Area (ft²)', 'Non-residential Area (ft²)'
]]

eui_chicago_df

,latitude,longitude,Residential EUI (kWh/m²/year),Non-residential EUI (kWh/m²/year),Residential Area (ft²),Non-residential Area (ft²)
3,41.887365,-87.653505,NaN,103.480835,NaN,104849.0
18,41.903424,-87.669405,342.622520,NaN,106888.0,NaN
24,41.881965,-87.626793,NaN,387.106660,NaN,240076.0
25,41.880924,-87.624934,890.944749,NaN,144036.0,NaN
28,41.792213,-87.599950,NaN,776.106261,NaN,64028.0
...,...,...,...,...,...,...
28322,41.910843,-87.629022,182.668913,NaN,61200.0,NaN
28324,41.874634,-87.630418,NaN,189.609700,NaN,1237455.0
28325,41.973128,-87.651076,NaN,206.961670,NaN,129029.0
28326,41.893483,-87.621931,370.701161,NaN,99530.0,NaN


In [7]:
eui_chicago_df.to_csv("../data/02_interim/eui_chicago.csv", index=False)